In [3]:
# import nest_asyncio
# nest_asyncio.apply()
import os
import django
import inspect

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from books.models import Book
from blog.models import Post
from books import utils

from asgiref.sync import sync_to_async
from django.shortcuts import get_object_or_404
from django.http import HttpRequest

In [6]:
from model_bakery import baker
from books.models import *
from books import models
clss = [i[1] for i in inspect.getmembers(models, inspect.isclass)]
clss
# for i in clss:
#     baker.make(i, _quantity=200)
Post.objects.first().title

'guMWwWPOBTioJRIYVimKABYhpDDktxOH'

In [7]:
vars(clss[1].objects.all()[0])

{'_state': <django.db.models.base.ModelState at 0x7f0643120c10>,
 'address_id': 606,
 'address': 'zGUdSHPuxrxirMaGlvoQxiNNdwJiHkhxWmsDbRNumdkauPugGz',
 'address2': None,
 'district': 'rIKUgZgtZXIhEjevocfh',
 'city_id': 601,
 'postal_code': None,
 'phone': 'mZuPvtdUTbkkzGarzSCO',
 'last_update': datetime.datetime(2023, 9, 25, 12, 33, 57, 995257)}

In [7]:
b = Book.objects.raw("select * from books_book as t where title=$$%s$$" % '11ef11b1d80aa8e0')
# from django.core.serializers import serialize
# serialize('json', b)


[]

In [ ]:
from books.utils import get_sql_searchparams
get_sql_searchparams(Book, ['col1', 'col2'], 'plant')